In [1]:
using Pkg
pkg"activate ."
pkg"instantiate"

 Activating environment at `~/Documents/otimizacao-em-julia/notebooks/Project.toml`


In [2]:
using Plots, Interact, LinearAlgebra, ForwardDiff
gr(size=(600,400))
contour(1:10, 1:10, atan)
nothing

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-13498060478557746961\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-11431379963165331115\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-13498060478557746961\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [3]:
ℓ = zeros(2)
u = ones(2)
x = randn(2)
x, clamp.(x, ℓ, u)

([0.8684541165292377, -0.4324896849337499], [0.8684541165292377, 0.0])

In [8]:
using ForwardDiff, LinearAlgebra

function caixas(f, ℓ, u, x)
    x = copy(clamp.(x, ℓ, u))
    n = length(x)
    
    ∇f(x) = ForwardDiff.gradient(f, x)
    H(x) = ForwardDiff.hessian(f, x)
    
    WL = findall(x .≈ ℓ)
    WU = findall(x .≈ u)
    
    @info("", WL, WU)

    gx = ∇f(x)
    # ∇f(x) = zₗ - zᵤ
    zL = zeros(n)
    zU = zeros(n)
    zL[WL] = gx[WL]
    zU[WU] = -gx[WU]

    t₀ = time()
    Δt = 0.0
    iter = 0
    resolvido = norm(gx - zL + zU) < 1e-8 && all(zL .≥ 0) && all(zU .≥ 0)
    cansado = Δt > 10.0 || iter > 1000
    
    while !(resolvido || cansado)
        Hx = H(x)
        IZ = setdiff(1:n, WL, WU)
        Z = Matrix(1.0I, n, n)[:,IZ]
        d = -Z * ( (Z' * Hx * Z) \ (Z' * gx) )
        if all(ℓ .≤ x + d .≤ u)
            @info("Andei 100% do passo")
            x += d
            if norm(d) < 1e-8
                @info("Encontrei um estacionario")
                zL .= 0
                zU .= 0
                zL[WL] = gx[WL]
                zU[WU] = -gx[WU]
                if any(zL .< 0)
                    i = findfirst(zL .< 0)
                    WL = setdiff(WL, i)
                elseif any(zU .< 0)
                    i = findfirst(zU .< 0)
                    WU = setdiff(WU, i)
                end
            end
        else
            # ℓ ≤ x + t * d ≤ u
            # dᵢ > 0 : t ≤ (uᵢ - xᵢ) / dᵢ
            # dᵢ < 0 : t ≤ (ℓᵢ - xᵢ) / dᵢ
            t = 1.0
            @info("Encontrei uma face")
            primeiro = 0
            for i = 1:n
                if d[i] > 0
                    α = (u[i] - x[i]) / d[i]
                    if t > α
                        t = α
                        primeiro = -i
                    end
                elseif d[i] < 0
                    α = (ℓ[i] - x[i]) / d[i]
                    if t > α
                        t = α
                        primeiro = i
                    end
                end
            end
            @assert primeiro != 0
            if primeiro > 0
                WL = [WL; primeiro]
            else
                WU = [WU; -primeiro]
            end
            x += t * d
        end
        
        @info("$iter", x, gx, d, zL, zU, WL, WU)
        gx = ∇f(x)
        Δt = time() - t₀
        iter += 1
        resolvido = norm(gx - zL + zU) < 1e-8 && all(zL .≥ 0) && all(zU .≥ 0)
        cansado = Δt > 10.0 || iter > 10
    end
    
    return x, gx, zL, zU, Δt, iter, resolvido, cansado
end

caixas (generic function with 1 method)

In [9]:
f(x) = (x[1] - 2)^2 + 4 * (x[2] - 2)^2
ℓ = -ones(2)
u = ones(2)
x = -ones(2)
x, gx, zL, zU, _ = caixas(f, ℓ, u, x)

┌ Info: 
│   WL = [1, 2]
│   WU = Int64[]
└ @ Main In[8]:13
┌ Info: Andei 100% do passo
└ @ Main In[8]:34
┌ Info: Encontrei um estacionario
└ @ Main In[8]:37
┌ Info: 0
│   x = [-1.0, -1.0]
│   gx = [-6.0, -24.0]
│   d = [0.0, 0.0]
│   zL = [-6.0, -24.0]
│   zU = [0.0, 0.0]
│   WL = [2]
│   WU = Int64[]
└ @ Main In[8]:81
┌ Info: Encontrei uma face
└ @ Main In[8]:55
┌ Info: 1
│   x = [1.0, -1.0]
│   gx = [-6.0, -24.0]
│   d = [3.0, 0.0]
│   zL = [-6.0, -24.0]
│   zU = [0.0, 0.0]
│   WL = [2]
│   WU = [1]
└ @ Main In[8]:81
┌ Info: Andei 100% do passo
└ @ Main In[8]:34
┌ Info: Encontrei um estacionario
└ @ Main In[8]:37
┌ Info: 2
│   x = [1.0, -1.0]
│   gx = [-2.0, -24.0]
│   d = [0.0, 0.0]
│   zL = [0.0, -24.0]
│   zU = [2.0, 0.0]
│   WL = Int64[]
│   WU = [1]
└ @ Main In[8]:81
┌ Info: Encontrei uma face
└ @ Main In[8]:55
┌ Info: 3
│   x = [1.0, 1.0]
│   gx = [-2.0, -24.0]
│   d = [0.0, 3.0]
│   zL = [0.0, -24.0]
│   zU = [2.0, 0.0]
│   WL = Int64[]
│   WU = [1, 2]
└ @ Main In[8]:81
┌ Inf

([1.0, 1.0], [-2.0, -8.0], [0.0, 0.0], [2.0, 8.0], 0.15515804290771484, 5, true, false)

In [10]:
using Logging

xg = range(-2, 2, length=50)
yg = range(-2, 2, length=50)
ℓ = -ones(2)
u = ones(2)

@manipulate for c1 in xg, c2 in xg, x01 = -1:1, x02 = -1:1
    f(x) = (x[1] - c1)^2 + 9 * (x[2] - c2)^2
    contour(xg, yg, (x,y) -> f([x;y]), leg=false)
    
    x0 = [x01; x02]
    scatter!([x01], [x02], c=:red, ms=3)
    plot!([ℓ[1], ℓ[1], u[1], u[1], ℓ[1]], [ℓ[2], u[2], u[2], ℓ[2], ℓ[2]], c=:black, lw=2)
    x, gx, zL, zU, _ = with_logger(NullLogger()) do
        caixas(f, ℓ, u, x0)
    end
    scatter!([x[1]], [x[2]], c=:blue, ms=4)
end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["c1"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 50,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Any} with 2 listeners. Value:
25, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/abel/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/abel/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/abel/.julia/packages/InteractBase/sOe2Z/src/../assets/all.js"), Asset("css", nothing, "/home/abel/.julia/packages/InteractBase/sOe2Z/src/../assets/style.css"), Asset("css", nothing, "/home/abel/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f01d22c0c40, Task (runnable) @0x00007f01d22c0c40), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"-2.0\",\"-1.91837\",\"-1.83673\",\"-1.7551\",\"-1.67347\",\"-1.59184\",\"-1.5102\",\"-1.42857\",\"-1.34694\",\"-1.26531\",\"-1.18367\",\"-1.10204\",\"-1.02041\",\"-0.938776\",\"-0.857143\",\"-0.77551\",\"-0.693878\",\"-0.612245\",\"-0.530612\",\"-0.44898\",